In [1]:
import pandas as pd
import numpy as np

In [2]:
path_ast = "./astral/PXD046453_duplicates.parquet"
path_tof = "./timsTOF/tof_train.parquet"
path_pro = "./proteome_tools/proteome_tools_0.parquet"

In [3]:
df_ast = pd.read_parquet(path_ast)
df_tof = pd.read_parquet(path_tof)
df_pro = pd.read_parquet(path_pro)

In [4]:
dataset_cols = [
    'prosit_sequence',
    'charge',
    'collision_energy',
    'method_nr',
    'machine',
    'intensities_raw',
]

# set collision_energy_aligned_normed as collision_energy
df_tof['collision_energy'] = df_tof['collision_energy_aligned_normed']

# temporarily norm astral collision energy
df_ast['collision_energy'] = df_ast['collision_energy'] / 100

df_ast = df_ast[dataset_cols]
df_tof = df_tof[dataset_cols]
df_pro = df_pro[dataset_cols]

In [5]:
df_ast = df_ast.sample(frac=1, random_state=42).reset_index(drop=True)
df_pro = df_pro.sample(frac=1, random_state=42).reset_index(drop=True)
df_tof = df_tof.sample(frac=1, random_state=42).reset_index(drop=True)

rows_per_dataset = 210_000

df_combined = pd.concat([df_ast.head(rows_per_dataset), df_tof.head(rows_per_dataset), df_pro.head(rows_per_dataset)], axis=0, ignore_index=True)
df_combined

df_combined['intensities_raw'] = df_combined['intensities_raw'].apply(lambda x: x.astype(np.float64))

df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
df_combined.head(100)

,prosit_sequence,charge,collision_energy,method_nr,machine,intensities_raw
0,FHCEICNVK,2,0.29000,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.05357467637..."
1,YGLAVFQAFLR,2,0.29000,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.05016706145..."
2,YVENPSQVLNCER,2,0.30000,CID,Astral,"[0.15839938819408417, 0.0, -1.0, 0.0, 0.0, -1...."
3,DLATQICKQQSPDA,2,0.28000,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.02944250871..."
4,ALLAMYTNQAEQCR,2,0.30000,CID,Astral,"[0.4077594578266144, 0.0, -1.0, 0.0, 0.0, -1.0..."
...,...,...,...,...,...,...
95,LFLFPGESAQNEATSYHQK,3,0.21069,HCD,Orbitrap_Fusion_Lumos,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
96,LHAVPNLR,2,0.32000,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.00548584043..."
97,KPVKSLSLPGLRAHL,3,0.33000,HCD,TOF,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
98,VPVSVNLLSK,2,0.30000,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.02543801406..."


In [7]:
method_map = {
    'CID': 0,
    'HCD': 1,
}

machine_map = {
    'Astral': 0,
    'TOF': 1,
    'Orbitrap_Fusion_Lumos': 2
}

df_combined['charge_oh']    = df_combined['charge']   .apply(lambda x: np.eye(6)[x-1]           .astype(int))
df_combined['method_nr_oh'] = df_combined['method_nr'].apply(lambda x: np.eye(2)[method_map[x]] .astype(int))
df_combined['machine_oh']   = df_combined['machine']  .apply(lambda x: np.eye(3)[machine_map[x]].astype(int))

df_combined['modified_sequence'] = df_combined['prosit_sequence']

In [26]:
df_combined

,prosit_sequence,charge,collision_energy,method_nr,machine,intensities_raw,charge_oh,method_nr_oh,machine_oh,modified_sequence
0,FEFHHGDYEK,3,0.34,HCD,TOF,"[0.0012383208003038033, 0.0, 0.0, 0.0, 0.0, 0....","[0, 0, 1, 0, 0, 0]","[0, 1]","[0, 1, 0]",FEFHHGDYEK
1,GRDDITVHDNIK,2,0.28,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.03308445538...","[0, 1, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",GRDDITVHDNIK
2,VTSSGVLLLDNYSDR,2,0.30,CID,Astral,"[0.4873846769332886, 0.0, -1.0, 0.0, 0.0, -1.0...","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",VTSSGVLLLDNYSDR
3,LLESDYFR,2,0.31,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.02768777614...","[0, 1, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",LLESDYFR
4,LDSSDIYNELK,2,0.30,CID,Astral,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.41207116842...","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",LDSSDIYNELK
...,...,...,...,...,...,...,...,...,...,...
629995,DPETLVGYSMVGCQR,2,0.30,CID,Astral,"[0.07622350007295609, 0.0, -1.0, 0.0, 0.0, -1....","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",DPETLVGYSMVGCQR
629996,YLEKSGVL,1,0.32,HCD,TOF,"[0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, ...","[1, 0, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",YLEKSGVL
629997,LQHGTQQQDLNKK,3,0.32,HCD,TOF,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2686567164179...","[0, 0, 1, 0, 0, 0]","[0, 1]","[0, 1, 0]",LQHGTQQQDLNKK
629998,FMADIDLDPGCTLNK,2,0.30,CID,Astral,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.05899238213...","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",FMADIDLDPGCTLNK


In [27]:
df_combined.to_parquet("./combined_dlomix_format.parquet")

In [28]:
df_combined.head(10).to_parquet("./combined_dlomix_format_testing.parquet")

In [12]:
df_combined[df_combined['prosit_sequence'].str.len() >= 28].head(10).to_parquet("./combined_dlomix_format_testing2.parquet")

,prosit_sequence,charge,collision_energy,method_nr,machine,intensities_raw
88,KDPEGLFLQDNIVAEFSVDETGQmSATAK,3,0.334532,HCD,Orbitrap_Fusion_Lumos,"[0.4300000071525574, 0.0, 0.0, 0.8799999952316..."
678,NHGVVMPDANKENTLNQLVGAAFGAAGQR,3,0.213971,HCD,Orbitrap_Fusion_Lumos,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
679,TGLAGAPGPPGVKGSSGLPGSPGIQGPK,2,0.350000,CID,Orbitrap_Fusion_Lumos,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1...."
722,AGLPCQDLEFVQFHPTGIYGAGCLITEGCR,4,0.300000,CID,Astral,"[0.11255692690610886, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1571,EGSVVVDLAAEAGGNFETTKPGELYIHK,4,0.300000,CID,Astral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2667414844036..."
1635,ILLSQTTGVAIPLHASSLDDVSLASTPK,3,0.300000,CID,Astral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2109750658273..."
1939,PSHTLSSLATGASGGPPVSKAPTMDAQQDR,4,0.235426,HCD,Orbitrap_Fusion_Lumos,"[0.0476190485060215, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2825,LIAALSTPSQQVQESVASCLPPLVPAIK,3,0.300000,CID,Astral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3803,SSGATPVSGPPPPSASSTPAGQPTAVSR,2,0.317961,HCD,Orbitrap_Fusion_Lumos,"[0.03999999910593033, 0.0, -1.0, 0.0, 0.0, -1...."
4183,DLEEDREENPGLTSPEPQLPNSPTGVVGAK,3,0.385300,HCD,Orbitrap_Fusion_Lumos,"[0.10999999940395355, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [8]:
# split 80/10/10

n_rows = df_combined.shape[0]

train   = df_combined.iloc[                 :int(n_rows * 0.8)]
val     = df_combined.iloc[int(n_rows * 0.8):int(n_rows * 0.9)]
test    = df_combined.iloc[int(n_rows * 0.9):]

train.to_parquet("./combined_dlomix_format_train.parquet")
val  .to_parquet("./combined_dlomix_format_val.parquet")
test .to_parquet("./combined_dlomix_format_test.parquet")

In [12]:
test

,prosit_sequence,charge,collision_energy,method_nr,machine,intensities_raw,charge_oh,method_nr_oh,machine_oh,modified_sequence
567000,LLELGPKPEVAQQTR,3,0.30,CID,Astral,"[0.11873848736286163, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 0, 1, 0, 0, 0]","[1, 0]","[1, 0, 0]",LLELGPKPEVAQQTR
567001,ENIVEAIIHSPELIR,2,0.30,CID,Astral,"[0.45250797271728516, 0.0, -1.0, 0.0, 0.0, -1....","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",ENIVEAIIHSPELIR
567002,HDTVFGSYLYK,2,0.30,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.58220640569...","[0, 1, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",HDTVFGSYLYK
567003,DAMQYASESK,2,0.30,CID,Astral,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.19688180088...","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",DAMQYASESK
567004,YPLFEGQETGK,2,0.30,CID,Astral,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.10170126706...","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",YPLFEGQETGK
...,...,...,...,...,...,...,...,...,...,...
629995,FDTGNLCMVTGGANLGR,2,0.30,CID,Astral,"[0.13473941385746002, 0.0, -1.0, 0.0, 0.0, -1....","[0, 1, 0, 0, 0, 0]","[1, 0]","[1, 0, 0]",FDTGNLCMVTGGANLGR
629996,NIHSSISHESPAVK,2,0.27,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.02242943548...","[0, 1, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",NIHSSISHESPAVK
629997,IFSLLEK,2,0.32,HCD,TOF,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.24063553084...","[0, 1, 0, 0, 0, 0]","[0, 1]","[0, 1, 0]",IFSLLEK
629998,NPEHFVVCNR,3,0.30,CID,Astral,"[0.33678939938545227, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 0, 1, 0, 0, 0]","[1, 0]","[1, 0, 0]",NPEHFVVCNR
